In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler(storage='local')

checking local storage... done
we have total 447215 articles (275713 downloaded, 171502 trashed)


# 1. URL을 모은다

In [12]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [3]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (211.80 seconds): Collecting URLs... usatoday            

100.00% (0.57 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,26051,25981,2529
reuters,1120,1120,292
dailymail,2912,2912,248
euronews,1058,1058,183
investing.com,998,998,171
telegraph,808,808,114
yahoo,390,390,102
businessinsider,558,558,92
usatoday,747,747,85
rt,450,450,83


,url,pubs,actual_pub
0,https://abcnews.go.com/Politics/national-secur...,"abcnews, realclearpolitics",abcnews
1,https://businessinsider.co.za/sasol-culture-of...,"businessinsider, news24",businessinsider
2,https://businessinsider.co.za/south-african-se...,"businessinsider, news24",businessinsider
3,https://businessinsider.co.za/universe-expansi...,"businessinsider, news24",businessinsider
4,https://cbsnews.com/news/polyamory-relationshi...,"cbs, nationalreview, americanconservative",cbs
5,https://foxbusiness.com/real-estate/chuck-devo...,"fox, realclearpolitics",fox
6,https://foxbusiness.com/technology/pentagon-ha...,"fox, realclearpolitics",fox
7,https://foxnews.com/media/washington-post-al-b...,"fox, americanconservative",fox
8,https://foxnews.com/opinion/jim-hanson-baghdad...,"fox, realclearpolitics",fox
9,https://foxnews.com/opinion/sally-pipes-warren...,"fox, realclearpolitics",fox


# 2. 해당 URL의 뉴스를 다운로드한다

In [4]:
download_result = crawler.download(); download_result

20.40% (275.18 seconds): Downloading... metro               

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


29.62% (410.62 seconds): Downloading... thehill             

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100.00% (1906.90 seconds): Downloading... euronews            

collect                             download          \
                     collected collected_unique selected downloaded trashed   
all                      26051            25981     2529       1523    1006   
reuters                   1120             1120      292        124     168   
dailymail                 2912             2912      248        198      50   
euronews                  1058             1058      183         13     170   
investing.com              998              998      171         59     112   
telegraph                  808              808      114        109       5   
yahoo                      390              390      102         82      20   
businessinsider            558              558       92         62      30   
usatoday                   747              747       85         42      43   
rt                         450              450       83         16      67   
metro                      236              236       83         70      13   
aljazeera                  494              494       74         11      63   
news24                    1231             1228       56         53       3   
independent                733              733       50         50       0   
mirror                     207              207       49         47       2   
cnn                        825              825       45         22      23   
marketwatch                178              178       44         30      14   
breitbart                  401              401       40         38       2   
washingtonpost             133              133       40         40       0   
guardian                   165              165       32         27       5   
fox                        363              363       27         22       5   
axios                      120              116       27         21       6   
arynews                    138              138       27         11      16   
cbs                        420              420       26         10      16   
indiatimes                 179              179       25         19       6   
joongang                   361              361       24         24       0   
afr                        446              446       24         24       0   
thestreet                  133              133       23         10      13   
zdnet                      339              339       23         10      13   
bbc                        277              277       23         20       3   
...                        ...              ...      ...        ...     ...   
washingtontimes            454              454       13         13       0   
realclearpolitics          145              129       13          9       4   
nymag                      231              231       11          8       3   
theverge                  1187             1187       10         10       0   
thenextweb                  85               85        9          9       0   
cryptocoin                  44               44        8          8       0   
dailybeast                  50               49        7          2       5   
hotair                      85               84        7          5       2   
npr                        898              898        7          6       1   
techcrunch                  19               19        7          7       0   
cryptonews                 326              326        7          1       6   
nbcnews                    190              185        6          0       6   
huffpost                    70               70        5          5       0   
vice                        87               87        5          4       1   
atlantic                   103              103        5          5       0   
townhall                   609              608        4          4       0   
news.com.au                277              277        4          1       3   
newrepublic                 20               20        4          4      

In [28]:
download_result.to_dict()

{('collect', 'collected'): {'all': 8645,
  'reuters': 764,
  'fox': 299,
  'investing.com': 510,
  'cnn': 623,
  'telegraph': 542,
  'news24': 270,
  'marketwatch': 137,
  'washingtonpost': 137,
  'wsj': 204,
  'aljazeera': 168,
  'thestreet': 128,
  'washingtontimes': 115,
  'nationalreview': 730,
  'guardian': 164,
  'financialpost': 113,
  'msnbc': 136,
  'nbcnews': 87,
  'zdnet': 239,
  'businessinsider': 76,
  'cnbc': 66,
  'bbc': 192,
  'afr': 86,
  'vice': 105,
  'techcrunch': 20,
  'axios': 21,
  'politico': 206,
  'realclearpolitics': 139,
  'nytimes': 99,
  'theverge': 947,
  'scmp': 252,
  'thehill': 88,
  'huffpost': 65,
  'chinadaily': 393,
  'morningstar': 32,
  'rt': 16,
  'indiatimes': 33,
  'cryptonews': 327,
  'time': 19,
  'japan-news': 70,
  'townhall': 0,
  'arirang': 0,
  'yahoo': 16,
  'americanconservative': 0,
  'atlantic': 0,
  'arynews': 0,
  'usatoday': 0,
  'thenextweb': 0,
  'nymag': 0,
  'thinkprogress': 0,
  'arstechnica': 0,
  'newrepublic': 0,
  'npr':

Bigquery와 local 뉴스 갯수가 다를 것이다. Bigquery에 뉴스를 모두 업데이트 한 후 해당 repository에 문제가 생겨서, github에서 전일기준 데이터를 clone하여, 뉴스를 다시 다운로드 받은 날이 있다. 2019-10-22

In [15]:
275632+81

275713